In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import datetime

from tensorflow.keras.models import Sequential , load_model , Model
from tensorflow.keras.layers import Dense ,Conv2D , MaxPooling2D , AveragePooling2D , Flatten ,Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras import backend as k
from tensorflow.keras.callbacks import EarlyStopping , ModelCheckpoint


In [2]:
catImgs = cv2.imread("../input/cat-and-dog/test_set/test_set/cats/cat.4004.jpg")

plt.imshow(catImgs)
plt.show()

In [3]:
dogImgs = cv2.imread("../input/cat-and-dog/test_set/test_set/dogs/dog.4006.jpg")
plt.imshow(dogImgs)
plt.show()

In [4]:
trainDataGenerator = ImageDataGenerator()

trainDataGenerator = trainDataGenerator.flow_from_directory("../input/cat-and-dog/training_set/training_set/",
                                        target_size = (224,224),
                                        batch_size = 32,
                                        shuffle = True,
                                        class_mode = "binary")

In [5]:
testDataGenerator = ImageDataGenerator()
testDataGenerator = testDataGenerator.flow_from_directory("../input/cat-and-dog/test_set/test_set/",
                                                          target_size = (224,224),
                                                          batch_size = 32,
                                                          shuffle = False,
                                                          class_mode = "binary")

In [6]:
model = Sequential()

# 1st Layer
model.add(Conv2D(input_shape = (224, 224, 3), filters = 64, kernel_size = (3, 3), padding = "same", activation = "relu"))
model.add(Conv2D(filters = 64, kernel_size = (3, 3), padding = "same", activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))

# 2nd Layer
model.add(Conv2D(filters = 128, kernel_size = (3, 3), padding = "same", activation = "relu"))
model.add(Conv2D(filters = 128, kernel_size = (3, 3), padding = "same", activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))

#3rd Layer
model.add(Conv2D(filters = 256, kernel_size = (3, 3), padding = "same", activation = "relu"))
model.add(Conv2D(filters = 256, kernel_size = (3, 3), padding = "same", activation = "relu"))
model.add(Conv2D(filters = 256, kernel_size = (3, 3), padding = "same", activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))

#4th Layer
model.add(Conv2D(filters = 512, kernel_size = (3, 3), padding = "same", activation = "relu"))
model.add(Conv2D(filters = 512, kernel_size = (3, 3), padding = "same", activation = "relu"))
model.add(Conv2D(filters = 512, kernel_size = (3, 3), padding = "same", activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))

#5th Layer
model.add(Conv2D(filters = 512, kernel_size = (3, 3), padding = "same", activation = "relu"))
model.add(Conv2D(filters = 512, kernel_size = (3, 3), padding = "same", activation = "relu"))
model.add(Conv2D(filters = 512, kernel_size = (3, 3), padding = "same", activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), name = 'vgg16'))

model.add(Flatten(name = "flatten"))


model.add(Dense(256, activation = 'relu', name = 'fcN1'))
model.add(Dense(128, activation = 'relu', name = 'fcN2'))
model.add(Dense(1, activation = "sigmoid", name = 'output'))


In [7]:
model.summary()

In [8]:
vgg_models = Model(inputs = model.input, outputs = model.get_layer('vgg16').output)

In [9]:
vgg_models.load_weights("../input/vgg16-weight-metrics/vgg16_weights_tf_dim_ordering_tf_kernels_notop-2.h5")

In [10]:
for modelLayer in vgg_models.layers:
    modelLayer.trainable = False

In [11]:
for model_layer in model.layers:
    print(f"Layer : {modelLayer}\nTrainable Layer : {modelLayer.trainable}\n")

In [12]:
optimizer = SGD(learning_rate = 1e-4, momentum = 0.9)

In [13]:
model.compile(loss = "binary_crossentropy",
             optimizer = optimizer,
             metrics = ["accuracy"])

In [14]:
earlyStoping = EarlyStopping(monitor = 'val_accuracy',
                            mode = "max", 
                            verbose = 1, 
                            patience = 2)

In [16]:
models = model.fit(trainDataGenerator,
          validation_data = testDataGenerator,
          verbose = 1,
          callbacks = earlyStoping,
          epochs = 5)

In [17]:
models.params

In [18]:
models.history.keys()



In [19]:
acc = models.history['accuracy']
val_acc = models.history['val_accuracy']

loss = models.history['loss']
val_loss = models.history['val_loss']

In [20]:
EPOCHS = 5

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(range(EPOCHS), acc, label='Training Accuracy')
plt.plot(range(EPOCHS), val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(range(EPOCHS), loss, label='Training Loss')
plt.plot(range(EPOCHS), val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [21]:
model.save("Models1/CatsAndDogs.h5")

In [22]:
classData = ["cat" , "dog"]

In [28]:
testImageData = cv2.imread("../input/cat-and-dog/test_set/test_set/cats/cat.4008.jpg")
imageDataFeature = cv2.resize(testImageData,(244,244))
imageDataFeature = np.array([imageDataFeature])

In [29]:
imageDataFeature.shape

In [30]:
probability = model.predict(imageDataFeature)[0]

In [31]:
probaMax = probability.argmax(axis = 0)
probaMax

In [32]:
imageDataLabels = f"{classData[probaMax]} , {probability[probaMax]*100}"

cv2.putText(testImageData , imageDataLabels , (10,35) , cv2.FONT_HERSHEY_COMPLEX , 1.0, (0,255,0) ,3)

plt.imshow(testImageData , cmap=  "gray")
plt.show

In [ ]:
./Models1/CatsAndDogs.h5